In [228]:
import numpy as np
from scipy.stats import skewnorm, skew
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

def simulate_data(classes, n_vars, n, max_mu, max_sigma, max_skew):
    #The multivariate skew normal number generator
    def rng(mu, sigma, skew, n=1):
        k = len(mu)
        if not (k == len(sigma) and k ==len(skew)): 
            raise Exception("Mu, Sigma and Skew should be same length")

        data = np.zeros((int(n),k))

        for i in range(k):
            data[:,i] = skewnorm.rvs(skew[i], loc=mu[i], scale=sigma[i], size=int(n)) 

        return data
    
    if(np.sum(classes) != 1):
        raise Exception("Classes dont sum up to 1")
        
    n_classes = len(classes)
    sigma = np.random.randint(1,max_sigma,n_vars)
    skew = np.random.randint(-max_skew,max_skew,n_vars)
    mu =  np.random.randint(-max_mu, max_mu, (n_classes, n_vars))
    
    n_obs_class = np.round(np.dot(classes,n))
    
    data = np.zeros((int(np.sum(n_obs_class)),n_vars+1))
    for i in range(n_classes):
        #calculate indexes
        start = int(np.sum(n_obs_class[0:i]))
        end = int(np.sum(n_obs_class[0:i+1]))
        
        #set the data
        data[start:end,0] = i
        data[start:end,1:] = rng(mu[i,:], sigma, skew, n_obs_class[i])
        
    X = data[:,1:]
    y = data[:,0]
    
    columns = ["x"+str(x) for x in range(n_vars + 1)]
    columns[0] = "class"
    
    df = pd.DataFrame(data,columns=columns)
    df["class"] = df["class"].astype(int)
    return X,y, df



#parameters
classes = [0.2, 0.5, 0.3] #percentages
n_vars = 5
n = 100000
max_mu = 20
max_sigma = 10
max_skew = 7
np.random.seed(12345)

#generate data
X,y, df = simulate_data(classes, n_vars, n, max_mu, max_sigma, max_skew)

display(df.groupby(["class"]).agg(["count", "mean", "var"]))

#make train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


x1                          x2                           x3  \
       count       mean       var  count       mean        var  count   
class                                                                   
0      20000  -8.383487  3.514547  20000  11.728285  13.851757  20000   
1      50000  -9.338244  3.504340  50000  -5.280513  13.565434  50000   
2      30000 -15.334147  3.433368  30000  23.741372  13.795757  30000   

                              x4                           x5             \
           mean       var  count       mean        var  count       mean   
class                                                                      
0     -5.572087  1.529541  20000 -13.791497  17.219184  20000 -13.727987   
1     -4.575062  1.529570  50000  -4.801718  16.765843  50000  13.281707   
2      1.429721  1.504561  30000   6.185656  17.029445  30000   6.281662   

                  
             var  
class             
0      13.803013  
1      13.529252  
2      13.534370

In [ ]:
    
    

# display(np.cov(data.T))




# plt.plot(x, y, 'x')
# plt.axis('equal')
# plt.show()


# plt.hist(x)
# plt.hist(y)
# # plt.hist(z)

# from mpl_toolkits import mplot3d
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.scatter3D(x, y, z, c=z);

In [ ]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_lda = lda.fit_transform(X, y)

plt.xlabel('LD1')
plt.ylabel('LD2')
plt.scatter(
    X_lda[:,0],
    X_lda[:,1],
    c=y,
    cmap='rainbow',
    alpha=0.7,
    edgecolors='b'
)